# Introduction to the concept of retrieval augmented generation (RAG)

> *PLEASE NOTE: This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio. Also, you should have provisioned access in your AWS account for the required Bedrock models as described [here](https://catalog.us-east-1.prod.workshops.aws/event/dashboard/en-US/workshop/20-introduction/21-bedrock).*

---

Question Answering (QA) is an important task that involves extracting answers to factual queries posed in natural language. Typically, a QA system processes a query against a knowledge base containing structured or unstructured data and generates a response with accurate information. Ensuring high accuracy is key to developing a useful, reliable and trustworthy question answering system, especially for enterprise use cases. However, in this notebook, we will highlight a well-documented issue with LLMs: LLM's are unable to answer questions outside of their training data.

---
## Setup the `boto3` client connection to Amazon Bedrock

Similar to notebook "01_workshop_setup.ipynb", we will create a client side connection to Amazon Bedrock using the `boto3` library.

In [ ]:
%pip install -r ../requirements.txt

In [ ]:
from IPython.display import Markdown, display

import json
import os
import sys
from rich import print as rprint

import boto3
import botocore

module_path = "../.."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww
from utils.prompt_utils import prompts_to_messages

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

---
## Highlighting the Contextual Issue

We are trying to model a situation where we are asking the model to provide information about Amazon SageMaker Jumpstart foundation models. We will first ask the model based on the training data to provide us with an answer about pricing of this technoloy. This technique is called `Zero Shot`. Let's take a look at Claude's response to a quick question "How are SageMaker JumpStart foundation models priced?"

In [ ]:
import json
prompt = "What is the pricing model for SageMaker Jumpstart models?"


body = json.dumps({
    "max_tokens": 500,
    "messages": prompts_to_messages(prompt),
    "anthropic_version": "bedrock-2023-05-31"
})


modelId = "anthropic.claude-3-haiku-20240307-v1:0"

accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)

response_body = json.loads(response.get("body").read())

rprint(response_body.get("content")[0]["text"])

The answer provided by Claude would either be incorrect or Claude may indicate that it does not have the requisite information to answer the question. This is not surprising because SageMaker Jumpstart foundation models are a quite new technology at the time of writing, meaning that there are more likely changes to the correct answer to the question which are not included in Claude's training data.

This implies we need to augment the prompt with additional data about the desired technology question and then the model will return us a very factually accurate. We will see how this improves the response in the next section.

---
## Manually Providing Correct Context

In order to have Claude correctly answer the question provided, we need to provide the model context which is relevant to the question. Below is a frequently asked question (FAQ) from the public SageMaker documentation. 

```
Question:

How are SageMaker JumpStart foundation models priced?

Answer:

For proprietary models, you are charged for software pricing determined by the model provider and SageMaker infrastructure charges based on the instance used. For publicly available models, you are charged SageMaker infrastructure charges based on the instance used. For more information, see Amazon SageMaker Pricing and the AWS Marketplace.
```

We can inject this context into the prompt as shown below and ask the LLM to answer our question based on the context provided.

In [ ]:
prompt = '''Answer question provided below by using the context provided. Do not use any information other than what is provided in the context. If the context is insufficient, please respond with "Insufficient information".

<context>
SageMaker JumpStart Pricing:
For proprietary models, you are charged for software pricing determined by the model provider and SageMaker infrastructure charges based on the instance used. 
For publicly available models, you are charged SageMaker infrastructure charges based on the instance used. For more information, see Amazon SageMaker Pricing and the AWS Marketplace.
</context>

Question: What is the pricing model for SageMaker Jumpstart models?

'''

body = json.dumps({
    "max_tokens": 256,
    "messages": prompts_to_messages(prompt),
    "anthropic_version": "bedrock-2023-05-31"
})


response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
rprint(response_body.get("content")[0]["text"])

Now you can see that the model answers the question accurately based on the factual context. However, this context had to be added manually to the prompt. In a production setting, we need a way to automate the retrieval of this information.

## Providing External Context Automatically
In practice, a RAG solution would dynamically provide the relevant context to the LLM. This is done by performing a search over a large corpus of documents to find the most relevant information to the question. Then providing the relevant context to the LLM along with the question. This is a powerful technique that allows the LLM to answer questions that are not in its training data.

In subsequent sections, you will learn how to build your own search engine, but here will illustrate the RAG concept using Wikipedia search. Wikipedia is a commonly used data source for training LLMs, so we will ask a question about a recent event that would not be in the training data. 

In [ ]:
%pip install wikipedia
%pip install wikipedia-api

In [ ]:
import wikipedia
import wikipediaapi

wiki_wiki = wikipediaapi.Wikipedia('RAGexample','en')

query = "Who won the Super Bowl in 2024?"

search_results = wikipedia.search(query)
page_content = wiki_wiki.page(search_results[0]).text

prompt = f'''Use the context provided to answer the question below. If the context is insufficient, please respond with "Insufficient information".

<context>
{page_content}
</context>

Question: {query}
'''

body = json.dumps({
    "max_tokens": 256,
    "messages": prompts_to_messages(prompt),
    "anthropic_version": "bedrock-2023-05-31"
})



response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

rprint(response_body.get("content")[0]["text"])

---
## Quick Note: Long Context Windows

One known limitation for RAG based solutions is the need for inclusion of lots of text into a prompt for an LLM. Fortunately, Claude can help this issue by providing an input token limit of 200k tokens. This limit [corresponds to around 150k words](https://www.anthropic.com/news/claude-2-1) which is an astounding amount of text.

Let's take a look at an example of Claude handling this large context size...

In [ ]:
book = ''
with open('../data/book/book.txt', 'r') as f:
    book = f.read()
print('Context:', book[0:53], '...')
print('The context contains', len(book.split(' ')), 'words')

In [ ]:
prompt =f'''

Summarize the plot of this book.

<book>
{book}
</book>

'''

body = json.dumps({
    "max_tokens": 1000,
    "messages": prompts_to_messages(prompt),
    "anthropic_version": "bedrock-2023-05-31"
})

# response = boto3_bedrock.invoke_model(
#     body=body, modelId='anthropic.claude-instant-v1', accept='application/json', contentType='application/json'
# )

response = boto3_bedrock.invoke_model(
    body=body, modelId='anthropic.claude-3-sonnet-20240229-v1:0', accept='application/json', contentType='application/json'
)
response_body = json.loads(response.get('body').read())
rprint(response_body.get("content")[0]["text"])

---
## Next steps

Now you have been able to see a concrete example where LLMs can be improved with correct context injected into a prompt, lets move on to the next notebook to see how we can automate this process using OpenSearch vector database.